In [6]:
from google.colab import files
yuklenen = files.upload()


Saving input.txt to input (1).txt


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# ——— Sabitler ———
toplu_boyut        = 32      # Aynı anda işlenecek dizi sayısı
blok_uzunlugu      = 8       # Modelin görebileceği maksimum bağlam uzunluğu
maks_tekrar        = 3000    # Eğitim iterasyonu
deger_araligi      = 300     # Kaç adımda bir doğrulama yapılacak
ogrenme_orani      = 1e-2    # AdamW öğrenme hızı
cihaz              = 'cuda' if torch.cuda.is_available() else 'cpu'
deger_tekrar       = 200     # Kayıpsız değerlendirme adımları

torch.manual_seed(1337)


In [ ]:

with open('input.txt', 'r', encoding='utf-8') as dosya:
    tam_metin = dosya.read()

# Benzersiz karakterler ve eşlemeler
karakterler   = sorted(list(set(tam_metin)))
kelime_sayisi = len(karakterler)

char2idx = {ch: i for i, ch in enumerate(karakterler)}
idx2char = {i: ch for i, ch in enumerate(karakterler)}

kodla   = lambda s: [char2idx[c] for c in s]
coz     = lambda l: ''.join([idx2char[i] for i in l])


In [ ]:
tam_tensor = torch.tensor(kodla(tam_metin), dtype=torch.long)
ayrim      = int(0.9 * len(tam_tensor))        # %90 eğitim, %10 doğrulama

egitim_veri = tam_tensor[:ayrim]
dogrulama   = tam_tensor[ayrim:]


In [ ]:
def mini_paket(bolum):
    """Eğitim veya doğrulama için (x,y) çiftleri üretir."""
    veri = egitim_veri if bolum == 'train' else dogrulama
    rast_idx = torch.randint(len(veri) - blok_uzunlugu, (toplu_boyut,))
    x = torch.stack([veri[i:i+blok_uzunlugu]     for i in rast_idx])
    y = torch.stack([veri[i+1:i+blok_uzunlugu+1] for i in rast_idx])
    return x.to(cihaz), y.to(cihaz)

@torch.no_grad()
def kayip_tahmin():
    sonuc = {}
    model.eval()
    for kisim in ['train', 'val']:
        kayiplar = torch.zeros(deger_tekrar)
        for k in range(deger_tekrar):
            X, Y = mini_paket(kisim)
            _, l = model(X, Y)
            kayiplar[k] = l.item()
        sonuc[kisim] = kayiplar.mean()
    model.train()
    return sonuc


In [ ]:
class BasitBigramDilModeli(nn.Module):
    def __init__(self, sozluk_boyu):
        super().__init__()
        # Her sembol, bir sonraki sembolün logit’ini doğrudan gören embedding tablo
        self.gomme = nn.Embedding(sozluk_boyu, sozluk_boyu)

    def forward(self, girdi, hedef=None):

        logit = self.gomme(girdi)

        if hedef is None:
            kayip = None
        else:
            B, T, C = logit.shape
            logit  = logit.view(B*T, C)
            hedef  = hedef.view(B*T)
            kayip  = F.cross_entropy(logit, hedef)

        return logit, kayip

    def uret(self, baslangic, yeni_token_sayisi):
        """Belirli sayıda yeni karakter üretir."""
        for _ in range(yeni_token_sayisi):
            tahmin, _ = self(baslangic)
            tahmin = tahmin[:, -1, :]
            olasilik = F.softmax(tahmin, dim=-1)
            sonraki = torch.multinomial(olasilik, 1)
            baslangic = torch.cat((baslangic, sonraki), dim=1)
        return baslangic


In [ ]:
model = BasitBigramDilModeli(kelime_sayisi).to(cihaz)
optimizator = torch.optim.AdamW(model.parameters(), lr=ogrenme_orani)


In [ ]:
for adim in range(maks_tekrar):
    # Ara ara doğrulama kaybını yazdır
    if adim % deger_araligi == 0:
        k = kayip_tahmin()
        print(f"Adım {adim:4d} | Eğitim kaybı: {k['train']:.4f} | Doğrulama kaybı: {k['val']:.4f}")

    # Mini‑batch çek
    xb, yb = mini_paket('train')

    # Kaybı hesapla ve geriye yay
    logit, kayip = model(xb, yb)
    optimizator.zero_grad(set_to_none=True)
    kayip.backward()
    optimizator.step()


In [ ]:

bas_kontekst = torch.zeros((1, 1), dtype=torch.long, device=cihaz)
yeni = model.uret(bas_kontekst, yeni_token_sayisi=500)[0].tolist()
print(coz(yeni))
